## Importing required packages

In [2]:
#importing required libraries 

#Pandas is mainly used for data analysis.
import pandas as pd 

#it used for real time computer vision
import cv2

# allows us to operate on underlying interpreter
import sys

#array computing
import numpy as np

#used in estimate the performance of machine lerning algorith 
from sklearn.model_selection import train_test_split 


In [3]:
#csv file directory 
csv_dir ='fer2013.csv'
#resize size of image
size_of_img = (48,48)

In [4]:
data = pd.read_csv(csv_dir)
print(len(data))

35887


In [5]:
# function which load the csv file and reshape the images into 48*48 image and detect the faces and 
#stores in faces array and finally returns the faces and emotions array

def read_csv():
    data = pd.read_csv(csv_dir)
    pixels = data['pixels'].tolist()
    width, height = size_of_img
    faces = []
    
    for pix_seq in pixels:
        face = [int(pix) for pix in pix_seq.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), size_of_img)
        faces.append(face.astype('float32'))
        
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
        
    return faces, emotions

In [6]:
#currently data in an int8 formate so it need to convert its type to float32 before feed in to the network. 
#and also rescaele the pixels values in range 0 - 1 .
def input_preprocess(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

In [7]:
faces, emotions = read_csv()
faces = input_preprocess(faces)
xtrain, xtest, ytrain, ytest = train_test_split(faces,emotions,test_size = 0.2, shuffle=True)

In [8]:
print('Train Images: ',xtrain.shape, ytrain.shape)
print('Test Imgaes: ',xtest.shape,ytest.shape)

Train Images:  (28709, 48, 48, 1) (28709, 7)
Test Imgaes:  (7178, 48, 48, 1) (7178, 7)


In [9]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.layers import Activation, Convolution2D, Conv2D, Dropout, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Input, MaxPooling2D, SeparableConv2D
from keras.models import Model
from keras.regularizers import l2

#parameters
batch_size = 32 # the num. of samples to eork through before updating the internal model parameters.
num_of_epochs = 100 #the num. of times that the learning algorithm will work through the entire training dataset.
shape_of_img = (48, 48, 1) #shape of the image to 48 x 48 x 1
verbose = True # to see the training progress for each epochs
num_of_classes = 7
patience = 50
base_dir = 'models/' #model directory 
l2_regularization = 0.01

## Using CNN Model 

In [10]:


#data generator
data_generator = ImageDataGenerator(featurewise_center = False,featurewise_std_normalization = False,
                                    rotation_range = 10, width_shift_range = 0.1, height_shift_range = 0.1,
                                    zoom_range = .1, horizontal_flip = True)
#model parameters 
regularization = l2(l2_regularization)

# base
image_input = Input(shape_of_img)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(image_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(filters=8, kernel_size=(3,3), strides=(1,1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# module 1
# residual module 
residual = Conv2D(filters=16, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=16, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 2
# residual module 
residual = Conv2D(filters=32, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=32, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 3
# residual module 
residual = Conv2D(filters=64, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

# module 4
# residual module 
residual = Conv2D(filters=128, kernel_size=(1,1), strides=(2,2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)

x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(filters=128, kernel_size=(3,3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(x)
x = layers.add([x,residual])

x = Conv2D(filters=num_of_classes, kernel_size=(3,3), padding='same')(x)
x = GlobalAveragePooling2D()(x)

output = Activation('softmax', name='predictions')(x)



#
model = Model(image_input, output)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 46, 46, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 46, 46, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 46, 46, 8)    0           batch_normalization[0][0]        
______________________________________________________________________________________________

## need to uncomment to train the data

In [30]:
# callbacks
''''log_file_path = base_dir + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_dir + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
 
model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_of_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))'''

Epoch 1/100
897/897 [==============================] - 243s 268ms/step - loss: 1.8974 - accuracy: 0.2811 - val_loss: 1.6430 - val_accuracy: 0.4069

Epoch 00001: val_loss improved from inf to 1.64299, saving model to models\_mini_XCEPTION.01-0.41.hdf5
Epoch 2/100
897/897 [==============================] - 278s 309ms/step - loss: 1.5652 - accuracy: 0.4161 - val_loss: 1.4953 - val_accuracy: 0.4408

Epoch 00002: val_loss improved from 1.64299 to 1.49528, saving model to models\_mini_XCEPTION.02-0.44.hdf5
Epoch 3/100
897/897 [==============================] - 268s 299ms/step - loss: 1.4441 - accuracy: 0.4572 - val_loss: 1.5074 - val_accuracy: 0.4557

Epoch 00003: val_loss did not improve from 1.49528
Epoch 4/100
897/897 [==============================] - 251s 279ms/step - loss: 1.3419 - accuracy: 0.4974 - val_loss: 1.5246 - val_accuracy: 0.4869

Epoch 00004: val_loss did not improve from 1.49528
Epoch 5/100
897/897 [==============================] - 240s 268ms/step - loss: 1.2904 - accuracy

897/897 [==============================] - 215s 239ms/step - loss: 0.9040 - accuracy: 0.6658 - val_loss: 1.0304 - val_accuracy: 0.6265

Epoch 00078: val_loss did not improve from 1.01401
Epoch 79/100
897/897 [==============================] - 215s 240ms/step - loss: 0.8968 - accuracy: 0.6644 - val_loss: 1.0951 - val_accuracy: 0.6020

Epoch 00079: val_loss did not improve from 1.01401
Epoch 80/100
897/897 [==============================] - 216s 240ms/step - loss: 0.8928 - accuracy: 0.6697 - val_loss: 1.0865 - val_accuracy: 0.6155

Epoch 00080: val_loss did not improve from 1.01401
Epoch 81/100
897/897 [==============================] - 215s 240ms/step - loss: 0.8875 - accuracy: 0.6698 - val_loss: 1.0456 - val_accuracy: 0.6255

Epoch 00081: val_loss did not improve from 1.01401
Epoch 82/100
897/897 [==============================] - 216s 240ms/step - loss: 0.8901 - accuracy: 0.6728 - val_loss: 1.0481 - val_accuracy: 0.6254

Epoch 00082: val_loss did not improve from 1.01401
Epoch 83/100


## To save the train model log 

In [44]:
#save the train model
#next time we do not need to train above train files again
#it will continue from above save model
#model.save("train_model.h5py") #uncomment to save 

INFO:tensorflow:Assets written to: train_model.h5py\assets


## Importing required packages

In [14]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import imutils
import cv2
import sys 

In [15]:
base_dir = 'models/'
detection_model_path = 'haarcascade/haarcascade_frontalface_default.xml'
emotion_recognition_model_path = base_dir + '_mini_XCEPTION.100-0.64.hdf5'



In [16]:
face_detection = cv2.CascadeClassifier(detection_model_path)

In [17]:
emotion_classifier = load_model(emotion_recognition_model_path)

In [18]:
emotions = ['angry', 'disgust', 'scared', 'happy', 'sad', 'surprised', 'neutral']

## Emotion dectection through web cam


In [22]:
cv2.namedWindow('facial_emotion_recognition')
camera = cv2.VideoCapture(0)  ## uncomment to use your laptop camera 
#camera = cv2.VideoCapture('various_emotions.mp4')  # uncomment to read from a video file

sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))

fourcc = cv2.VideoWriter_fourcc(*'mpeg')

out = cv2.VideoWriter()
out.open('output_various_facial_emotions.mp4',fourcc, 15, sz, True) # initialize the writer


# while True: # when reading from a video camera, use this while condition
while(camera.read()[0]):  # when reading from a video file, use this while condition
    color_frame = camera.read()[1]
    color_frame = imutils.resize(color_frame,width=min(720, color_frame.shape[1]))
    
    
    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)
    detected_faces = face_detection.detectMultiScale(gray_frame,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = color_frame.copy()    

    
    if len(detected_faces)>0:

        detected_faces = sorted(detected_faces, reverse=True, key=lambda x: (x[2]-x[0])*(x[3]-x[1]))[0] # if more than one faces
        (fx, fy, fw, fh) = detected_faces

        im = gray_frame[fy:fy+fh, fx:fx+fw]
        im = cv2.resize(im, (48,48))  # the model is trained on 48*48 pixel image 
        im = im.astype("float")/255.0
        im = img_to_array(im)
        im = np.expand_dims(im, axis=0)

        preds = emotion_classifier.predict(im)[0]
        emotion_probability = np.max(preds)
        label = emotions[preds.argmax()]

        cv2.putText(color_frame, label, (fx, fy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(color_frame, (fx, fy), (fx + fw, fy + fh),(0, 0, 255), 2)

    
    for (i, (emotion, prob)) in enumerate(zip(emotions, preds)):
        # construct the label text
        text = "{}: {:.2f}%".format(emotion, prob * 100)
        w = int(prob * 300)
        
        cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 50, 100), -1)
        cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
        cv2.putText(frameClone, label, (fx, fy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (100, 150, 100), 2)
        cv2.rectangle(frameClone, (fx, fy), (fx + fw, fy + fh), (100, 100, 100), 2)
    
    out.write(frameClone)
    out.write(canvas)
    
    cv2.imshow('emotion_recognition', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
camera.release()
out.release()
cv2.destroyAllWindows()

## Emotion Dection through Images

In [23]:
#https://github.com/priya-dwivedi/face_and_emotion_detection/blob/master/src/EmotionDetector_v2.ipynb
def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    #detect the face from image
    faces = face_detection.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    allfaces = []   
    rects = []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
        allfaces.append(roi_gray)
        rects.append((x,w,y,h))
    return rects, allfaces, img

img = cv2.imread("faces.jpg")
rects, faces, image = face_detector(img)
print('Number of face detected: ',len(faces))

i = 0

for face in faces:
    roi = face.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)

    # make a prediction on the ROI, then lookup the class
    predictions = emotion_classifier.predict(roi)[0]
    label = emotions[predictions.argmax()]   

    #Overlay our detected emotion on our pic
    position_of_labels = (rects[i][0] + int((rects[i][1]/10)), abs(rects[i][2] - 10))
    i += 1
    cv2.putText(image, label, position_of_labels , cv2.FONT_HERSHEY_SIMPLEX,0.8, (100,255,0), 1)
    
cv2.imshow("Facial Emotion Recogniser", image)
cv2.waitKey(0)

cv2.destroyAllWindows()

<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-23-3b39de3629c0>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Number of face detected:  24
